### Imports

In [2]:
import argparse
import os
import time

import dgl

import model
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn.functional as F
from dgl.data import CiteseerGraphDataset, CoraGraphDataset, PubmedGraphDataset
# from input_data import load_data

import torch
from dgl.data import DGLDataset

from preprocess import (
    mask_test_edges,
    mask_test_edges_dgl,
    preprocess_graph,
    sparse_to_tuple,
)
from sklearn.metrics import average_precision_score, roc_auc_score

os.environ["DGLBACKEND"] = "pytorch"

In [3]:
from tqdm import tqdm
import pandas as pd
import networkx as nx
import numpy as np

%load_ext autotime

time: 161 µs (started: 2023-06-08 19:22:36 -05:00)


In [4]:
import sys
sys.path.append("../")

from utils.utils import *

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 3.19 s (started: 2023-06-08 19:22:36 -05:00)


/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [5]:
torch.manual_seed(42)
np.random.seed(42)

time: 2.67 ms (started: 2023-06-08 19:22:39 -05:00)


### Parameters

In [6]:
import json

# dir = os.path.dirname(os.path.dirname(os.getcwd()))
dir = os.path.dirname(os.getcwd())
print(dir)

# opening JSON file
file = open("{}/parameters.json".format(dir))
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = "vgae"
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    for group in groups_id:
        subgroups_id[group] = [option]
    print("Subgroups id:\t", subgroups_id)

/home/ealvarez/Project/GNN_Filter/GNN_unsupervised
Exp:		 exp5
Method:		 vgae
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 dyn
Subgroups id:	 {'WT': ['dyn'], 'zwf1^': ['dyn'], 'pck1^': ['dyn']}
time: 3.7 ms (started: 2023-06-08 19:22:39 -05:00)


### Node embeddings

In [7]:
# custom dataset

class CustomDataset(DGLDataset):
    def __init__(self, name, nodes_data, edges_data):
        self.dir = dir
        self.nodes_data = nodes_data
        self.edges_data = edges_data
        super().__init__(name=name)
       
    def process(self):
        # node_features = torch.from_numpy(self.nodes_data["degree"].to_numpy())
        node_features = torch.from_numpy(np.log10(self.nodes_data["degree"].to_numpy()))
        node_features = node_features.to(torch.float32)
        node_features = torch.reshape(node_features, (-1, 1))

        # node_labels = torch.from_numpy(self.nodes_data["id"].to_numpy())
        # node_labels = node_labels.to(torch.float32)

        edge_features = torch.from_numpy(self.edges_data["weight"].to_numpy())
        edges_src = torch.from_numpy(self.edges_data["source"].to_numpy())
        edges_dst = torch.from_numpy(self.edges_data["target"].to_numpy())

        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=self.nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        # self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = self.nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

time: 2.33 ms (started: 2023-06-08 19:22:39 -05:00)


In [8]:
nodes_data = pd.read_csv("{}/output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(dir, exp, groups_id[0], subgroups_id[groups_id[0]][0]))
edges_data = pd.read_csv("{}/output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(dir, exp, groups_id[0], subgroups_id[groups_id[0]][0]))

dataset = CustomDataset("g1", nodes_data, edges_data)
graph = dataset[0]

print(graph)

Graph(num_nodes=31150, num_edges=12257101,
      ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
time: 2.45 s (started: 2023-06-08 19:22:40 -05:00)


In [9]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

parser = argparse.ArgumentParser(description="Variant Graph Auto Encoder")
parser.add_argument(
    "--learning_rate", type=float, default=1e-3, help="Initial learning rate."
)
parser.add_argument(
    "--epochs", "-e", type=int, default=300, help="Number of epochs to train."
)
parser.add_argument(
    "--hidden1",
    "-h1",
    type=int,
    default=32,
    help="Number of units in hidden layer 1.",
)
parser.add_argument(
    "--hidden2",
    "-h2",
    type=int,
    default=3,
    help="Number of units in hidden layer 2.",
)
parser.add_argument(
    "--datasrc",
    "-s",
    type=str,
    default="dgl",
    help="Dataset download from dgl Dataset or website.",
)
parser.add_argument(
    "--dataset", "-d", type=str, default="cora", help="Dataset string."
)
parser.add_argument("--gpu_id", type=int, default=0, help="GPU id to use.")
args = parser.parse_args("")


# check device
device = torch.device(
    "cuda:{}".format(args.gpu_id) if torch.cuda.is_available() else "cpu"
)
# device = "cpu"
device
# roc_means = []
# ap_means = []

device(type='cuda', index=0)

time: 7.12 ms (started: 2023-06-08 19:22:42 -05:00)


In [10]:
def compute_loss_para(adj):
    pos_weight = (adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = (
        adj.shape[0]
        * adj.shape[0]
        / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
    )
    weight_mask = adj.view(-1) == 1
    weight_tensor = torch.ones(weight_mask.size(0)).to(device)
    weight_tensor[weight_mask] = pos_weight
    return weight_tensor, norm


def get_acc(adj_rec, adj_label):
    labels_all = adj_label.view(-1).long()
    preds_all = (adj_rec > 0.5).view(-1).long()
    accuracy = (preds_all == labels_all).sum().float() / labels_all.size(0)
    return accuracy


def get_scores(edges_pos, edges_neg, adj_rec):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    adj_rec = adj_rec.cpu()
    # Predict on test set of edges
    preds = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]].item()))

    preds_neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]].data))

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

def train_vgae(graph, args, method, group, subgroup):
    # extract node features
    feats = graph.ndata.pop("feat").to(device)
    in_dim = feats.shape[-1]

    # generate input
    adj_orig = graph.adj_external().to_dense()

    # build test set with 10% positive links
    (
        train_edge_idx,
        val_edges,
        val_edges_false,
        test_edges,
        test_edges_false,
    ) = mask_test_edges_dgl(graph, adj_orig)

    graph = graph.to(device)

    # create train graph
    train_edge_idx = torch.tensor(train_edge_idx).to(device)
    train_graph = dgl.edge_subgraph(graph, train_edge_idx, relabel_nodes=False)
    train_graph = train_graph.to(device)
    adj = train_graph.adj_external().to_dense().to(device)

    # compute loss parameters
    weight_tensor, norm = compute_loss_para(adj)

    # create model
    vgae_model = model.VGAEModel(in_dim, args.hidden1, args.hidden2)
    vgae_model = vgae_model.to(device)

    # create training component
    optimizer = torch.optim.Adam(vgae_model.parameters(), lr=args.learning_rate)
    print(
        "Total Parameters:",
        sum([p.nelement() for p in vgae_model.parameters()]),
    )

    # create training epoch
    for epoch in tqdm(range(args.epochs)):
        t = time.time()

        # Training and validation using a full graph
        vgae_model.train()

        logits = vgae_model.forward(graph, feats)

        # compute loss
        loss = norm * F.binary_cross_entropy(
            logits.view(-1), adj.view(-1), weight=weight_tensor
        )
        kl_divergence = (
            0.5
            / logits.size(0)
            * (
                1
                + 2 * vgae_model.log_std
                - vgae_model.mean**2
                - torch.exp(vgae_model.log_std) ** 2
            )
            .sum(1)
            .mean()
        )
        loss -= kl_divergence

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # train_acc = get_acc(logits, adj)

        # val_roc, val_ap = get_scores(val_edges, val_edges_false, logits)

        # Print out performance
        """ print(
            "Epoch:",
            "%04d" % (epoch + 1),
            "train_loss=",
            "{:.5f}".format(loss.item()),
            "train_acc=",
            "{:.5f}".format(train_acc),
            "val_roc=",
            "{:.5f}".format(val_roc),
            "val_ap=",
            "{:.5f}".format(val_ap),
            "time=",
            "{:.5f}".format(time.time() - t),
        ) """

    """ test_roc, test_ap = get_scores(test_edges, test_edges_false, logits)
    # roc_means.append(test_roc)
    # ap_means.append(test_ap)
    print(
        "End of training!",
        "test_roc=",
        "{:.5f}".format(test_roc),
        "test_ap=",
        "{:.5f}".format(test_ap),
    ) """

    embeds = vgae_model.encoder(graph, feats)
    embeds = embeds.cpu().detach()

    df_node_embeddings = pd.DataFrame(data=embeds)
    df_node_embeddings

    # save
    df_node_embeddings.to_csv("{}/output/{}/node_embeddings/node-embeddings_{}_{}_{}.csv".format(dir, exp, method, group, subgroup), index=True)
    # print("Save node embeddings")

time: 2.62 ms (started: 2023-06-08 19:22:42 -05:00)


In [11]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

time: 404 µs (started: 2023-06-08 19:22:42 -05:00)


In [12]:
# get node embeddings

for group in tqdm(groups_id):
    for subgroup in tqdm(subgroups_id[group]):
        nodes_data = pd.read_csv("{}/output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(dir, exp, group, subgroup))
        edges_data = pd.read_csv("{}/output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(dir, exp, group, subgroup))

        # read dataset
        # data = load_data(args)
        data = CustomDataset("g_{}_{}".format(group, subgroup), nodes_data, edges_data)
        graph = data[0]

        # train
        train_vgae(graph, args, method, group, subgroup)

  0%|          | 0/3 [00:13<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.62 GiB (GPU 0; 10.75 GiB total capacity; 428.12 MiB already allocated; 2.94 GiB free; 430.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

time: 13.8 s (started: 2023-06-08 19:22:42 -05:00)


In [ ]:
df_node_embeddings = pd.read_csv("{}/output/{}/node_embeddings/node-embeddings_{}_{}_{}.csv".format(dir, exp, method, groups_id[0], 
                                                                                                    subgroups_id[groups_id[0]][0]), index_col=0)
df_node_embeddings.head()

,0,1,2
0,-1.253404,0.090255,-0.488362
1,1.288619,0.063895,0.204877
2,-1.191579,1.008257,1.362132
3,-0.368521,-0.067463,0.107074
4,0.384290,1.547227,0.986131


time: 10.9 ms (started: 2023-06-08 19:22:06 -05:00)
